In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.tree import DecisionTreeClassifier
import joblib

In [2]:
#Read data
df = pd.read_csv('/home/edward/Homelab/Machine_learning/Drug_prediction/Data/drugsComTest_complete.csv', sep=',')

#Create mask variable to define and process data to required parameters, in this case to keep all values with rating 10 as True. The mask results categorize as True or False, and keeps all True. Apply the mask to the DataFrame and save it as a new DataFrame.
mask = df["rating"] == 10
df_mask = df[mask]

#Extract specific data for independent variable within the DataFrame columns.
X = df_mask[['condition', "usefulCount"]]

#Replace name with numbers, by creating labels for the independent variable "condition".
#Import Sklearn LabelEncoder to encode the independent variable "condition".
from sklearn import preprocessing
le_condition = preprocessing.LabelEncoder()
le_condition.fit(X["condition"])
X["condition"] = le_condition.transform(X["condition"])


#Extract specific data for dependent variable within the DataFrame columns.
Y = df_mask[['drugName']]

#Replace name with numbers - Labels
le_drugN = preprocessing.LabelEncoder()
le_drugN.fit(Y['drugName'])
Y['drugName'] = le_drugN.transform(Y['drugName'])

#Split data into train and test
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=0)


"""
TRAIN MODEL
"""
#Create model
model = DecisionTreeClassifier(criterion='entropy')
model.fit(X_train, Y_train)


"""
PREDICT
"""
#Predict
Y_pred = model.predict(X_test)
Y_pred_proba = model.predict_proba(X_test)
Y_pred_proba = Y_pred_proba[:,1]


"""
EVALUATE MODEL
"""
#Evaluate model
from sklearn.metrics import confusion_matrix, accuracy_score, f1_score, precision_score, recall_score
acc = accuracy_score(Y_test, Y_pred)
print("Accuracy:", acc)



"""
SAVE MODEL
"""
#Save model
#joblib.dump(model, r'C:\\Users\\gaibo\\OneDrive\\Escritorio\\Python Projects\\Medicine\\Tree_Predictor_Model\\model.pkl')



/tmp/ipykernel_1424/2952274390.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X["condition"] = le_condition.transform(X["condition"])
/tmp/ipykernel_1424/2952274390.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Y['drugName'] = le_drugN.transform(Y['drugName'])


Accuracy: 0.1392479435957697


'\nSAVE MODEL\n'